### Now  we have the image data with its preprocessing done and ready for filtering for feature engineering.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from google.colab import drive
drive.mount('/drive')
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
import cv2

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [2]:
img = np.load('/drive/My Drive/ML Data/ImageNumpyTemp.npy')
grey = np.load('/drive/My Drive/ML Data/GreyImg.npy')

In [3]:
num,rows,cols,ch = img.shape
X = np.zeros((num,10000))
print(X.shape)
print(grey.shape)
print(img.shape)
'''
for i in range(0,num):
  f1 = img[i,:,:,:]*255               # f1 is for gabor filter
  f1 = f1.astype(np.uint8)
  
  # Code from here for the Gabor filter
  
  feature1 = feature1.astype(np.float32)
  feature1 = feaature1/255

  f2 = grey[i,:,:]*255                # f2 is for Sobel filter.
  f2 = f2.astype(np.uint8)
  edges = cv2.Canny(f2,100,200)
  feature2 = edges.ravel()
  feature2 = feature2.astype(np.float32)
  feature2 = feature2/255
  X[i] = np.concatenate((feature1, feature2), axis=None)
'''
for i in range(0,num):
  f2 = grey[i,:,:]*255                # f2 is for Sobel filter.
  f2 = f2.astype(np.uint8)
  edges = cv2.Canny(f2,100,200)
  feature2 = edges.ravel()
  feature2 = feature2.astype(np.float32)

  X[i,:] = feature2/255
  
print(X.shape)

(23705, 10000)
(23705, 100, 100)
(23705, 100, 100, 3)
(23705, 10000)


In [4]:
y = np.load('/drive/My Drive/ML Data/Gender.npy')       # Loading Gender data
y = y.ravel()
print(y.shape)

(23705,)


In [5]:
Y = y

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=30, stratify=Y)

pca = PCA(n_components=3)

# Linear model
svm_l = SVC(kernel = 'linear')
pipeline_l = Pipeline([("pca", pca), ("svm_l", svm_l)])

parameters = {'SVM__C':[0.001, 0.1, 100, 10e5]}
grid_l = GridSearchCV(pipeline_l, parameters, cv=5)
grid_l.fit(X_train, y_train)
y_pred_l = grid_l.predict(X_test)
acc_l = accuracy_score(y_test,y_pred_l)
print('Test Accuracy Score for linear kernel: ' + str(acc_l))

# Sigmoid Model
svm_s = SVC(kernel='sigmoid')
pipeline_s = Pipeline([("pca", pca), ("svm_s", svm_s)])

parameters = {'SVM__C':[0.001, 0.1, 100, 10e5], 'SVM__gamma':[10,1,0.1,0.01]}
grid_s = GridSearchCV(pipeline_s, parameters, cv=5)
grid_s.fit(X_train, y_train)
y_pred_s = grid_s.predict(X_test)
acc_s = accuracy_score(y_test,y_pred_s)
print('Test Accuracy Score for sigmoid kernel : ' + str(acc_s))

# rbf Model
svm_rbf = SVC(kernel = 'rbf')
pipeline_rbf = Pipeline([("pca", pca), ("svm_rbf", svm_rbf)])

parameters = {'SVM__C':[0.001, 0.1, 100, 10e5], 'SVM__gamma':[10,1,0.1,0.01]}
grid_rbf = GridSearchCV(pipeline_rbf, parameters, cv=5)
grid_rbf.fit(X_train, y_train)
for param in grid_rbf.get_params().keys():
    print(param)
y_pred_rbf = grid_rbf.predict(X_test)
acc_rbf = accuracy_score(y_test,y_pred_rbf)
print('Test Accuracy Score for rbf kernel: ' + str(acc_rbf))

# Polynomial model

svm_poly = SVC(kernel = 'poly')
pipeline_poly = Pipeline([("pca", pca), ("svm_poly", svm_poly)])

parameters = {'SVM__C':[0.001, 0.1, 100, 10e5], 'SVM__gamma':[10,1,0.1,0.01]}
grid_poly = GridSearchCV(pipeline_poly, parameters, cv=5)
grid_poly.fit(X_train, y_train)
y_pred = grid_poly.predict(X_test)
acc_poly = accuracy_score(y_test,y_pred_poly)
print('Test Accuracy Score for polynomial kenel: ' + str(acc_poly))

ValueError: ignored

In [ ]:
# Considering img in same format as x.
def detect(img):  
  y_pred_rbf = grid_rbf.predict(img)
  y_pred_poly = grid_poly.predict(img)
  y_pred_l = grid_l.predict(img)
  y_pred_s = grid_s.predict(img)

  per = (y_pred_rbf[0]*acc_rbf + y_pred_poly[0]*acc_poly + y_pred_l[0]*acc_l + y_pred_s[0]*acc_s)/4

  if per < 0.5:
    per = 1 - per
    return 'Male', per
  elif per > 0.5:
    return 'Female', per
  else :
    return 'Not sure' 


i =  10000                      # To be inputted by user.

RealImg = X[i] 
res,per = detect(RealImg)
RealImgShow = img[i,:,:,:]*255
RealImgShow = RealImgShow.astype(np.uint8)
RealImgSho = cv2.cvtColor(RealImgShow,cv2.COLOR_BGR2RGB) 

print("The Image is : \n")
plt.imshow(RealImgSho)
plt.show()


print("Its predicted that the person shown is ", res," with ", per*100 , " % surity !")
